In [37]:
import pandas as pd

In [38]:
##Model training

In [39]:
df=pd.read_csv('./data/gemstone.csv')
df.head()


,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [40]:
df=df.drop(labels=['id'],axis=1)

In [41]:
##independed and dependent features
X=df.drop(labels=['price'],axis=1)
y=df[['price']]

In [42]:
##Define  which columns should be ordinal-encoded and which should be scaled 
categorical_cols=X.select_dtypes(include='object').columns
numerical_cols=X.select_dtypes(exclude='object').columns

In [43]:
### Define the custom ranking for each ordinal variable
cut_categories=['Fair','Good','Very Good','Premium','Ideal']
color_categories=['D','E','F','G','H','I','J']
clarity_categories=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [44]:
from sklearn.impute import SimpleImputer

In [53]:
from sklearn.impute import SimpleImputer ## handling the missing value 
from sklearn.preprocessing import StandardScaler ## handling the feature scaling 
from  sklearn.preprocessing import OrdinalEncoder ## give the rank on categorical feature
#Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [54]:
##Numerical_pipeline
num_pipeline=Pipeline(
    steps=[('imputer',SimpleImputer(strategy='median')),
           ('scaler',StandardScaler())
           
           ]
)

## Categorical_pipeline
cat_pipeline=Pipeline(
    steps=[('imputer',SimpleImputer(strategy='most_frequent')),
           ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
           ('scaler',StandardScaler())
           ]

           
)


preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)


])





In [55]:
#### Train test split 
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [56]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [57]:
## Model Training 
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error


In [58]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [61]:
regression.coef_

array([[ 6432.97591819,  -132.34206204,   -70.48787525, -1701.38593925,
         -494.17005097,   -76.32351645,    68.80035873,  -464.67990411,
          652.10059539]])

In [62]:
regression.intercept_

array([3976.8787389])

In [74]:
import numpy as np
def evaluate_model(true,prediction):
    mae=mean_absolute_error(true,prediction)
    mse=mean_squared_error(true,prediction)
    rmse=np.sqrt(mean_squared_error(true,prediction))
    r2_square=r2_score(true,prediction)
    return mae,rmse,r2_square

In [75]:
### Train multiple models
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
model_list=[]
r2_list=[]

for i in range (len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    ###Make predictions
    y_pred=model.predict(X_test)
    mae,rmse,r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model training performance')
    print('RMSE:',rmse)
    print('MAE:',mae)
    print('R2 score',r2_square)

    r2_list.append(r2_square)
    print('='*35)
    print('\n')



LinearRegression
Model training performance
RMSE: 1014.6296630375463
MAE: 675.075827006748
R2 score 0.9362906819996049


Lasso
Model training performance
RMSE: 1014.6591302750638
MAE: 676.2421173665508
R2 score 0.9362869814082755


Ridge
Model training performance
RMSE: 1014.6343233534445
MAE: 675.1077629781357
R2 score 0.9362900967491627


Elasticnet
Model training performance
RMSE: 1533.3541245902313
MAE: 1060.9432977143008
R2 score 0.8544967219374031


